In [3]:
import torch
from monai.networks.nets import UNet
from helpers import UNet_model
from models import CustomUNet
from torchviz import make_dot
from torchsummary import summary

2024-05-08 13:20:01.301006: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 13:20:01.379875: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 13:20:01.379955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 13:20:01.381801: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 13:20:01.396191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

In [8]:
Unet = UNet_model(2)
# summary(Unet, input_size=(1, 256, 256)) 

for name, module in Unet.named_modules():
    #x = module(x)
    print("Name: ", name)

Name:  
Name:  inc
Name:  inc.double_conv
Name:  inc.double_conv.0
Name:  inc.double_conv.1
Name:  inc.double_conv.2
Name:  inc.double_conv.3
Name:  inc.double_conv.4
Name:  inc.double_conv.5
Name:  down1
Name:  down1.maxpool_conv
Name:  down1.maxpool_conv.0
Name:  down1.maxpool_conv.1
Name:  down1.maxpool_conv.1.double_conv
Name:  down1.maxpool_conv.1.double_conv.0
Name:  down1.maxpool_conv.1.double_conv.1
Name:  down1.maxpool_conv.1.double_conv.2
Name:  down1.maxpool_conv.1.double_conv.3
Name:  down1.maxpool_conv.1.double_conv.4
Name:  down1.maxpool_conv.1.double_conv.5
Name:  down2
Name:  down2.maxpool_conv
Name:  down2.maxpool_conv.0
Name:  down2.maxpool_conv.1
Name:  down2.maxpool_conv.1.double_conv
Name:  down2.maxpool_conv.1.double_conv.0
Name:  down2.maxpool_conv.1.double_conv.1
Name:  down2.maxpool_conv.1.double_conv.2
Name:  down2.maxpool_conv.1.double_conv.3
Name:  down2.maxpool_conv.1.double_conv.4
Name:  down2.maxpool_conv.1.double_conv.5
Name:  down3
Name:  down3.maxpool_

In [4]:
outputs = {}
def get_activation(name):
    def hook(resUnet, input, output):
        outputs[name] = output
    return hook

In [4]:
resUnet = UNet(
            spatial_dims=2,
            in_channels=1,
            out_channels=2,
            channels=(64, 128, 256, 512, 1024),
            strides=(2, 2, 2, 2),
            num_res_units=2,
        )
# for name, module in resUnet.named_modules():
#     print(f"{name}: {module}")
summary(resUnet, input_size=(1, 256, 256)) 



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]             640
            Conv2d-2         [-1, 64, 128, 128]             640
    InstanceNorm2d-3         [-1, 64, 128, 128]               0
           Dropout-4         [-1, 64, 128, 128]               0
             PReLU-5         [-1, 64, 128, 128]               1
            Conv2d-6         [-1, 64, 128, 128]          36,928
    InstanceNorm2d-7         [-1, 64, 128, 128]               0
           Dropout-8         [-1, 64, 128, 128]               0
             PReLU-9         [-1, 64, 128, 128]               1
     ResidualUnit-10         [-1, 64, 128, 128]               0
           Conv2d-11          [-1, 128, 64, 64]          73,856
           Conv2d-12          [-1, 128, 64, 64]          73,856
   InstanceNorm2d-13          [-1, 128, 64, 64]               0
          Dropout-14          [-1, 128,

In [4]:
resUnet = UNet(
            spatial_dims=2,
            in_channels=1,
            out_channels=2,
            channels=(64, 128, 256, 512, 1024),
            strides=(2, 2, 2, 2),
            num_res_units=2,
        )

In [23]:
resUnet.model[1].submodule[:]

Sequential(
  (0): ResidualUnit(
    (conv): Sequential(
      (unit0): Convolution(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (adn): ADN(
          (N): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): PReLU(num_parameters=1)
        )
      )
      (unit1): Convolution(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (adn): ADN(
          (N): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
          (D): Dropout(p=0.0, inplace=False)
          (A): PReLU(num_parameters=1)
        )
      )
    )
    (residual): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (1): SkipConnection(
    (submodule): Sequential(
      (0): ResidualUnit(
        (conv): Sequential(
          (unit0): Convolution(
            (conv): Conv2d(128, 256, kernel_

In [7]:
for name, layer in resUnet.named_modules():
    if name == 'model.2.0':
        layer.register_forward_hook(get_activation(name))

outputs = {}
x = torch.randn(1, 1, 256, 256)
resUnet(x)

# Access the output
residual_unit_83_output = outputs['model.2.0']
print("Output of ResidualUnit-83:", residual_unit_83_output.shape)


Output of ResidualUnit-83: torch.Size([1, 2, 256, 256])


In [1]:
import torch
from monai.networks.nets import UNet
from helpers import UNet_model
from models import CustomUNet
from torchviz import make_dot
from torchsummary import summary

2024-05-08 13:27:18.502923: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 13:27:18.581296: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 13:27:18.581598: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 13:27:18.583819: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 13:27:18.598691: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

In [2]:
cusresUnet = CustomUNet(
            spatial_dims=2,
            in_channels=1,
            out_channels=2,
            channels=(64, 128, 256, 512, 1024),
            strides=(2, 2, 2, 2),
            num_res_units=2,
        )

In [3]:
x = torch.randn(1, 1, 256, 256)  # Adjust the input tensor size
out = cusresUnet(x)
print(out.shape)


torch.Size([1, 64, 128, 128])
